In [9]:
from bokeh.io import output_file, show, output_notebook
from bokeh.layouts import widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.embed import components
from datetime import date
from random import randint
from bokeh.plotting import figure

In [10]:
output_notebook()

Loading BokehJS ...

In [11]:
import pandas as pd
df = pd.read_csv('../data/headcount.csv', sep = '\t')
df.head()

,FT or PT,Month,Headcount
0,PT,Jun-21,902
1,FT,Jun-21,7510
2,PT,May-21,905
3,FT,May-21,7659
4,PT,Apr-21,906


In [12]:

def data_table(df):
        
    data = dict(df)
    source = ColumnDataSource(data)

    columns = [
            TableColumn(field="Month", title="Month"),
            TableColumn(field="FT or PT", title="FT or PT"),
            TableColumn(field="Headcount", title="Headcount"),
        
        ]
    data_table = DataTable(source=source, columns=columns, width=600, height=400)
    return data_table




In [13]:
show(data_table(df))


In [60]:
df2 = df.groupby(['Month']).sum().reset_index()
df2.head()

,Month,Headcount
0,Apr-21,8633
1,Aug-20,10835
2,Dec-20,9915
3,Feb-21,9397
4,Jan-21,9793


In [73]:
def bar_chart(df):
    # prepare some data
    month = list(df2['Month'].values)
    headcount = list(df2['Headcount'].values)

    # output to static HTML file

    # create a new plot with a title and axis labels
    source = ColumnDataSource(data=dict(df))
    p2 = figure(title="simple line example", x_range=month,)#)x_axis_label='month', y_axis_label='headcount')
    p2.sizing_mode = 'stretch_width'
    # add a line renderer with legend and line thickness
    p2.vbar(x = 'Month', top = 'Headcount', width =  0.5, legend_label="Temp.", line_width=2, color = 'blue',source = source)
    #p2.line(x, y, legend_label="Temp.", line_width=2,color = 'blue' )
    #p2.line(x, y, legend_label="Temp.", line_width=2,color = 'orange')


    # show the results
    show(p2,)

In [74]:
bar_chart(df)

In [ ]:
fig1.sizing_mode = 'scale_width'

NameError: name 'p' is not defined

In [34]:
x

['Jun-21',
 'Jun-21',
 'May-21',
 'May-21',
 'Apr-21',
 'Apr-21',
 'Mar-21',
 'Mar-21',
 'Feb-21',
 'Feb-21',
 'Jan-21',
 'Jan-21',
 'Dec-20',
 'Dec-20',
 'Nov-20',
 'Nov-20',
 'Oct-20',
 'Oct-20',
 'Sep-20',
 'Sep-20',
 'Aug-20',
 'Aug-20',
 'Jul-20',
 'Jul-20']